In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra (Mathematical Computations)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
# plt.rcParams={'figsize.figure': (15, 10)}

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

In [ ]:
df = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

df.shape

In [ ]:
# explore what the data looks like
df.head(3)

Abstract the index 0 (questions) from the data. This will enable us perform some calculations and plot some chart on the dataset

In [ ]:
question = df.iloc[0]
len(question)

In [ ]:
# check the question class
type(question)

we can make it a list using tolist() but we will come back to that. Moving ON... 

In [ ]:
# drop the 0th index from the data
df = df.drop(0, axis=0)
print(df.shape)

In [ ]:
# convert time in seconds to an interger
df['Time from Start to Finish (seconds)'] = df['Time from Start to Finish (seconds)'].astype('int32')

In [ ]:
# removing the outliers using the statistical method (atleast that's what i call it)
# i chose the time column because of the method i want to use for removing outliers and it the only column
# that has a mean value :)
mean = df['Time from Start to Finish (seconds)'].mean()
std = df['Time from Start to Finish (seconds)'].std()

cut_off = std * 3

lower, upper = mean - cut_off, mean + cut_off
new_df = df[(df['Time from Start to Finish (seconds)']<upper)& (df['Time from Start to Finish (seconds)']>lower)]

print(new_df.shape, df.shape)

Looks like approx 200 entries has been dropped Moving On...

In [ ]:
# convert question to list
questions = question.tolist()
questions[:10]

Exploring Question 2: **'What is your gender?'**

In [ ]:

new_df['Q2'].value_counts().plot(kind='bar', figsize=(10, 7))
plt.title('Gender Distribution plot', size=20)
plt.show()

In [ ]:
# in terms of %
age_percent = (new_df['Q2'].value_counts(normalize=True) * 100 ).round(2)
age_percent

approx 79% of respondents are Man and 19% are Woman. Only 1.3% chose prefer not to say.

This can be attributed to many things like
1. Females are not so much in the tech world
2. Approach used in conducting the Survey might be skewed
3. DS is still a new field and not so much people knows about etc
4. etc etc

Moving ON...

Exploring Time: Time 

In [ ]:
counts, bin_edges = np.histogram(new_df['Time from Start to Finish (seconds)'])
new_df['Time from Start to Finish (seconds)'].plot.hist(bins=bin_edges)
plt.title('Time Spent Distribution', size=20);plt.show()

Wow heavily Skewed why is that?

In [ ]:
new_df['Time from Start to Finish (seconds)'].describe()

Comparing the mean and 50% (median) already shows the data is highly skewed.

Computing the range also shows huge difference

In [ ]:
# i converted the result to minute from seconds by dividing by 60
print(f"The highest amount of time (minutes) taken by a respondent to fill the survey was :{new_df['Time from Start to Finish (seconds)'].max() / 60}")
print(f"The lowest amount of time (minutes) taken by a respondent to fill the survey was :{new_df['Time from Start to Finish (seconds)'].min() / 60}")

An entry was completed in less than a minute? WOW
investigating ....

Also another entry took more than 2 days to complete :shocked: :amazed:

My Theory is the entry(ies) that uses less than a minute to complete did not answer 80% of the survey question.

Veryfying....

In [ ]:
print(f"{len(new_df[new_df['Time from Start to Finish (seconds)'] < 60])} entries spends less than a minute")

Given the amount of questions asked in the survey 20 seconds to fill out all question was probably not realistic, we can dig deeper and see whether all others question were actually answered.

for me a more adequate survey would take you approx 5min given the number of questions and 5 min = 300seconds

so all answers that is less than this would be removed dropped

In [ ]:
new_df[new_df['Time from Start to Finish (seconds)'] < 60].isna().sum()

Most questions were not answered..

i decide to dropped them

Further analysis could be:
1. Which gender category is more?
2. What is the highest age category?

In [ ]:
# drop all rows (entries) were time spent is less than 60
len(new_df[new_df['Time from Start to Finish (seconds)'] > 60])

I think that 5 minutes to fill the survey would be an appropriate number, so any entry less than 5min (300 seconds) will be dropped

My idea is 5 minute (least/minimum) is a resonable time to complete the form

In [ ]:
new_df = new_df[new_df['Time from Start to Finish (seconds)'] > 300]

In [ ]:
new_df.shape

Now spending more than a day to fill the survey is somehow for me i will drop anything greater than 86400 seconds.

I don't mean to say the values as in answers to the questions are not correct, i just want to remove such instance from the dataset

In [ ]:
# removing entries with time spent greater than 86400
new_df = new_df[new_df['Time from Start to Finish (seconds)'] <= 86400]
print(new_df.shape[0])

About 3000 entries were dropped

In [ ]:
# Let look at the distribution of the times
plt.figure(figsize=(10,7))
_, bin_edges = np.histogram(new_df['Time from Start to Finish (seconds)'])
new_df['Time from Start to Finish (seconds)'].plot.hist(bins=bin_edges)
plt.xlabel('Time in Seconds', fontsize=13); plt.ylabel('Frequency', fontsize=13); 
plt.title('Time Distribution', size=20)
plt.show()

Most respondents spends 10,000 seconds to complete the form

In [ ]:
# convert seconds to minute (more reletable) yes?
new_df['Time from Start to Finish (minutes)'] = new_df['Time from Start to Finish (seconds)'] / 60

plt.figure(figsize=(10,7))
_, bin_edges = np.histogram(new_df['Time from Start to Finish (minutes)'])
new_df['Time from Start to Finish (minutes)'].plot.hist(bins=bin_edges)
plt.xlabel('Time in Minutes', fontsize=13); plt.ylabel('Frequency', fontsize=13); 
plt.title('Time Distribution', size=20)
plt.show()

looks like more than 70% of respondent spend less than 1hr

We can say that respondent that spend more than 500 minutes probably forgot they started the survey already and time was counting on

Exploring the SEX distribution Time spent (minutes) 

let see the distribution of time as spent by each sex category

In [ ]:
sex_time_avg = new_df.groupby('Q2')['Time from Start to Finish (minutes)'].mean().sort_values(ascending=False)
sex_time_avg

In [ ]:
sex_time_avg.plot.bar(figsize=(10,6), title='Average Time spent per Gender')
plt.xticks(rotation=45);plt.xlabel('Gender Category');plt.ylabel('Frequecy');plt.show()

On Average, Nonniary Category spends 61.5 minutes to complete the survey

Woman spends 44.5 minutes and Man spends 37 minutes

Exploring the Age Range Category

In [ ]:
new_df['Q1'].value_counts().plot.bar(figsize=(10,6))
plt.title('Age Range Category Distribution', size=20)
plt.xlabel('Age Range Category', fontsize=13)
plt.ylabel('Frequency', fontsize=13)
plt.show()

Generally, there are more younger respondents atleast in terms of age range

Let test this theory>>>> Thoery is older people spends the highest time to fill the survey

By older people Age category 60-69 upwards

investigating..... :laugh:

In [ ]:
age_per_time = pd.DataFrame(new_df.groupby(['Q1'])['Time from Start to Finish (minutes)'].mean().sort_values(ascending=False)\
                .reset_index(name='average time spent')).rename(columns={'Q1': 'age range'})

Well, the theory is not accepted, not much evidence to reject it.

From what is shown, old people spends the least amount of time on average 30 minutes AMAZING right?

Adding a Bar chart wont be a bad idea right?

In [ ]:
age_per_time.plot.bar(figsize=(10, 6), title='Time Spent Per Age Group', 
                      x='age range', y='average time spent');

Well, My Theory ain't quite right yes? of course Infact it is the exact opposite people with age range 60 above spends the smallest time on average to fill the survey. AMAZING 

Exploring Q3: In which country do you currently reside?

In [ ]:
new_df['Q3'].value_counts().plot(kind='bar', figsize=(10, 6))
plt.show()

India is the most represented country amongst respondents who filled the survey followed by USA

Other is a combination of many other countries

What is the Average time spent per country?

In [ ]:
country_avg_time = pd.DataFrame(new_df.groupby('Q3')['Time from Start to Finish (minutes)'].mean().sort_values(ascending=False)\
                    .reset_index(name='Average time spent')).rename(columns={'Q3': 'Country'})


Respondents from Peru spends the highest amount of time on Average while Tunisia followed closely

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=country_avg_time.head(10), x='Country', y='Average time spent', color='red')
plt.title('Top 10 Countries with their average time spent'); plt.xticks(rotation=45);plt.show()

On Average Belgium spends 13 minutes to complete the survey which is the lowest with Peru having 68.6 minutes has the highest time spent to complete the survey. Tunisia follows closely with 65.3 minutes and China 61.7 minutes

Exploring Gender Distribution per Country...

In [ ]:
country_per_gender = pd.crosstab(new_df['Q2'], new_df['Q3'])
# country_per_gender

country_per_gender dataframe can be used to answer question like how many Woman filled the survey from let say Nigeria

In [ ]:
print("how many Woman filled the survey from Nigeria?")
print(f"{country_per_gender['Nigeria']['Woman']} Women")

it can be used to get the gender distribution per country wrt to kaggle. Limitations is it just a survey other factors needs to be considered as well

Another kind of dataframe that could be generated would be to answer questions like 

Which country has the highest number of young or old people? etc

In [ ]:
# country per age range distribution
country_per_age_range = pd.crosstab(new_df['Q1'], new_df['Q3'])
# country_per_age_range

In [ ]:
print(f"{country_per_age_range['India']['60-69']}")

Exploring Q4: What is the highest level of formal education that you have attained or plan to attain within the next 2 years?

In [ ]:
(new_df['Q4'].value_counts(normalize=True) * 100).round(2)

41% of the respondents wants to attain a Master's degree in 2 years or got it already

approx 35% wants a Bachelor's degree and 12% wants a Doctoral Degree

In [ ]:
new_df['Q4'].value_counts().plot.bar(figsize=(10,6), title='Level of Education in 2 years time')
plt.show()

Thank you for reading cheers